# Viterbi Algorithm for Nature Primer (3 Marks)

This notebook implements the Viterbi algorithm for a Hidden Markov Model (HMM),  
as described in the Nature Primer, to classify DNA sequences into Exon (E), Intron (I), or Splice Site (5).

We will:
1. Define the required HMM parameters.
2. Write a function to calculate the log-probability of a given state path.
3. Implement the full Viterbi algorithm to compute the most likely state sequence.


In [1]:
# Importing Libraries
# ---------------------------
import numpy as np
import math


## Step 1: Define Model Parameters

We define:
- Hidden states: E (Exon), 5 (Splice site), I (Intron)
- Observations: A, C, G, T
- Initial probabilities
- Transition probabilities
- Emission probabilities


In [2]:
# HMM Parameters - States and Observations
# ------------------------------------------------
# States: Exon (E), Intron (I), and a splice site (5)
states = ['E', '5', 'I']

# Observations: Nucleotides (A, C, G, T)
nucleotides = ['A', 'C', 'G', 'T']

# Initial probabilities for each state
initial_probabilities = {'E': 1.0, '5': 0.0, 'I': 0.0}


# Transition matrix: Probabilities of moving from one state to another
transition_probabilities = {
    'Start': {'E': 1.0, '5': 0.0, 'I': 0.0, 'End': 0.0},
    'E': {'E': 0.9, '5': 0.1, 'I': 0.0, 'End': 0.0},
    '5': {'E': 0.0, '5': 0.0, 'I': 1.0, 'End': 0.0},
    'I': {'E': 0.0, '5': 0.0, 'I': 0.9, 'End': 0.1}
}

# Emission probabilities: Probability of emitting nucleotides (A, C, G, T) in each state
emission_probs = {
    'E': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
    '5': {'A': 0.05, 'C': 0.00, 'G': 0.95, 'T': 0.00},
    'I': {'A': 0.4, 'C': 0.1, 'G': 0.1, 'T': 0.4},
}






## Step 2: Compute Log Probability of a Given Path

This function takes a state path and an observed nucleotide sequence,  
and returns the log-probability of that path producing the sequence.


In [3]:
# Safe Logarithm Function
# -------------------------------
# Utility function to compute logarithms safely
def safe_log(x):
    return -math.inf if x == 0 else math.log(x)


In [4]:
# Function to compute the log probability of a state path emitting an observed sequence
def get_log_prob_of_a_given_path(state_path, observed_sequence):
    log_prob = 0.0
    if len(state_path) != len(observed_sequence):
        raise ValueError("The length of state path and the observed sequence must be the same")
    prev_state = 'Start'
    for i in range(len(observed_sequence)):
        current_state = state_path[i]
        observed_state = observed_sequence[i]
        log_prob += safe_log(transition_probabilities[prev_state][current_state]) + \
                    safe_log(emission_probs[current_state][observed_state])
        prev_state = current_state
    if prev_state == 'I':
        log_prob += safe_log(transition_probabilities[prev_state]['End'])
    return log_prob



In [ ]:
# state_path = "EEEEEEEEEEEEEEEEEEE5IIIIIII"
state_path = "EEEEEEEEEEEEEEEEEE5IIIIIII"
observed_sequence = "CTTCATGTGAAAGCAGACGTAAGTCA"
ans = get_log_prob_of_a_given_path(state_path, observed_sequence)
print(ans)


-41.21967768602254


## Step 3: Viterbi Algorithm

This function implements the dynamic programming Viterbi algorithm  
to find the most probable sequence of hidden states that could generate a given DNA sequence.


In [6]:
# Implementation of the Viterbi algorithm
def viterbi_algo(observed_sequence):
    num_states = len(states)
    num_observations = len(observed_sequence)
    viterbi_matrix = np.full((num_states, num_observations), -np.inf)
    backpointers = np.zeros((num_states, num_observations), dtype=int)
    state_to_index = {s: i for i, s in enumerate(states)}

    # Initialize the DP table
    for i, state in enumerate(states):
        viterbi_matrix[i, 0] = safe_log(initial_probabilities[state]) + \
                               safe_log(emission_probs[state][observed_sequence[0]])

    # Recursive computation
    for t in range(1, num_observations):
        for curr_idx, curr_state in enumerate(states):
            max_log_prob = -np.inf
            best_prev_idx = 0
            for prev_idx, prev_state in enumerate(states):
                log_prob = viterbi_matrix[prev_idx, t - 1] + \
                           safe_log(transition_probabilities[prev_state][curr_state])
                if log_prob > max_log_prob:
                    max_log_prob = log_prob
                    best_prev_idx = prev_idx
            viterbi_matrix[curr_idx, t] = max_log_prob + \
                                          safe_log(emission_probs[curr_state][observed_sequence[t]])
            backpointers[curr_idx, t] = best_prev_idx

    # Backtracking to find the best path
    best_path = []
    best_final_idx = np.argmax(viterbi_matrix[:, -1])
    best_path.append(states[best_final_idx])

    for t in range(num_observations - 1, 0, -1):
        best_final_idx = backpointers[best_final_idx, t]
        best_path.insert(0, states[best_final_idx])

    return best_path, np.max(viterbi_matrix[:, -1])


In [7]:
# Example Usage of Viterbi Algorithm
# -------------------------------------------
observed_sequence = "CTTCATGTGAAAGCAGACGTAAGTCA"
best_path, log_prob = viterbi_algo(observed_sequence)
print(f"Most probable path is {best_path}")
print(f"Log probability of the path is {log_prob}")


Most probable path is ['E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E']
Log probability of the path is -38.677666280562796
